Vamos a hacer el análisis de la tabla para todas las features... Carguemos *TODAS* las tablas de los features

In [2]:
%matplotlib inline

import pylab
import config
import logging
import glob
import pandas as pd
from pandasql import sqldf
from social import relevant_variables
from IPython.display import display
from analyzers import correlation_analysis 
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

tables = {}

def add_deltas(df):
    pysqldf = lambda q: sqldf(q, locals())

    mean_table = df.groupby(by=["session", "speaker"]).mean()
    var_clause = ",".join(
        ["df.%s" % column for column in ["session", "speaker", "task"]] + 
        ["df.%s, mean.%s as mean_%s" % (var, var, var) for var in relevant_variables+["abs_entrainment", "entrainment"]])
    df_with_means = sqldf("""SELECT %s
        from df INNER JOIN mean_table as mean ON 
        df.session = mean.session AND 
        df.speaker = mean.speaker""" % var_clause, locals())
    
    for var in relevant_variables + ["abs_entrainment", "entrainment"]:
        df_with_means["delta_%s" % var] = df_with_means[var] - df_with_means["mean_%s" % var]
    
    return df_with_means

    

for path in glob.glob("./tables/*.csv"):
    var_name = path[9:-4]
    new_df = pd.DataFrame.from_csv(path)
    new_df['abs_entrainment'] = abs(new_df['entrainment'])
    
    tables[var_name] = add_deltas(new_df)
    

Primero calcularemos la correlación entre el `entrainment` y las distintas variables sociales, sin restar ninguna media


# Efectos fijos

Cada sesión y speaker nos define un "panel".

Acá agregamos las variables delta para cada variable social y el entrainment

In [8]:
from analyzers import correlation_analysis

delta_vars = ["delta_%s" % var for var in relevant_variables]

def full_correlation_analysis(df, entrainment_var="entrainment", social_vars=relevant_variables):
    ret = {}
    for ap_var in df:
        ret[ap_var]=correlation_analysis(df[ap_var], entrainment_var, social_vars)
    return pd.DataFrame(ret)

full_correlation_analysis(tables, entrainment_var="delta_abs_entrainment", social_vars=delta_vars)

,ENG_MAX,ENG_MEAN,F0_MAX,F0_MEAN,NOISE_TO_HARMONICS_RATIO,PHONEMES_AVG,PHONEMES_COUNT,SOUND_VOICED_LOCAL_SHIMMER,SYLLABES_AVG,SYLLABES_COUNT,VCD2TOT_FRAMES
delta_contributes_to_successful_completion,0.009178,0.112496,0.130004,0.171077,0.131616,0.097442,0.032698,0.070560,0.043844,0.014726,0.038566
delta_making_self_clear,0.284146,0.162805,0.112412,0.128231,0.231229,0.112094,0.101542,0.009196,0.121149,0.140586,-0.062272
delta_engaged_in_game,0.073708,0.149642,0.112006,0.177648,0.021502,0.064234,0.017154,0.046752,0.098592,0.029579,0.026382
delta_planning_what_to_say,0.074908,0.051997,0.008462,0.096622,-0.028353,0.046056,0.065626,-0.087426,0.053154,0.065397,0.059195
delta_gives_encouragement,0.070832,0.055857,0.080199,0.001122,0.121498,0.012637,-0.094421,0.059794,0.032240,-0.104907,0.094580
delta_difficult_for_partner_to_speak,-0.147741,0.094718,-0.073180,-0.110435,-0.040265,-0.120837,-0.077043,0.059470,0.039286,-0.102495,0.045296
delta_bored_with_game,0.045836,0.002184,-0.051501,-0.144783,0.125903,-0.012690,0.037535,-0.078760,-0.052685,0.051532,0.022578
delta_dislikes_partner,-0.098404,0.111599,0.027793,0.007620,0.082547,0.032566,-0.058673,-0.048746,0.034682,-0.050914,-0.085494


In [ ]:
00